#### This script is a testing ground for different types of machine learning models and their accuracy on predicting the time it would take to run the circuit in question.

In [1]:
from qiskit import *  # Import necessary modules from Qiskit
import pandas as pd
from qiskit import transpile
import random
import csv
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import random
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

from qiskit import IBMQ, Aer  # Import IBMQ and Aer from Qiskit
from qiskit.providers.aer.noise import NoiseModel  # Import NoiseModel from Qiskit's Aer provider
provider = IBMQ.load_account()  # Load IBM Quantum Experience account credentials
# IBMQ.load_account()
provider = IBMQ.get_provider(hub='********', group='********', project='main')  # Set the desired hub, group, and project
backend = provider.get_backend('ibm_perth')  # Get the backend named 'ibm_perth' from the provider

qubitCount = 7  # Define the number of qubits

noise_model = NoiseModel.from_backend(backend)  # Create a noise model from the backend's properties

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

Gates = ['rz', 'rx', 'ry', 'sx', 'x', 'y', 'z', 'h', 'cx', 'swap']  # Define a list of gate names
gateCosts = [1, 1, 1, 1, 1, 1, 1, 2, 5, 11]  # Define a list of gate costs corresponding to the gate names


C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (186064.81242819485 > 2 * 89549.87196202418). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (186064.81242819485 > 2 * 89549.87196202418). Truncating to maximum value.
  warn("Device model returned an invalid T_2 relaxation time greater than"
C:\Users\taren\anaconda3\lib\site-packages\qiskit\providers\aer\noise\device\models.py:264: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (186064.81242819485 > 2 * 89549.87196202418). Truncating to maximum value

In [2]:
def ComplexityFinder(qc):
    df = QCtoDF(qc)  # Convert the quantum circuit (qc) to a DataFrame (df)
    depth = depthFinder(df)  # Calculate the depth of the circuit using a depthFinder function
    
    # Get Cost
    cost = 0  # Initialize the cost variable to 0
    
    # Iterate over each gate in the 'Gate' column of the DataFrame
    for i in df['Gate']:
        if i in Gates:  # Check if the gate is in the predefined list of gate names
            cost += int(gateCosts[Gates.index(i)])  # Add the corresponding gate cost to the total cost
        else:
            cost += int(gateCosts[Gates.index(i[:2])])  # If the gate name is longer than 2 characters, consider only the first two characters
    
    return cost, depth  # Return the calculated cost and depth of the circuit


In [3]:
def QCtoDF(qc):
    string = qc.qasm()  # Convert the quantum circuit (qc) to a QASM string
    circuit = string.split(';')  # Split the string at each semicolon to get individual instructions
    circuit = circuit[3:]  # Remove the first 3 elements of the circuit list (header and metadata)
    circuit.pop(len(circuit)-1)  # Remove the last element of the circuit list (empty line)
    
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:  # Open a temporary CSV file for writing
        writer = csv.writer(f)  # Create a CSV writer object
        writer.writerow(['Gate', 'Qubit'])  # Write the column headers
        
        for i in range(len(circuit)):
            circuit[i] = circuit[i].replace("\n", '')  # Remove newline characters from the instruction
            temp = circuit[i].split(' ')  # Split the instruction at each space
            
            # Process qubit information
            for j in range(len(temp[1])):
                 temp[1] = temp[1].replace("q[", '')  # Remove the "q[" prefix
                 temp[1] = temp[1].replace("]", '')  # Remove the "]" suffix
            
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)  # Write the gate and qubit information as a row in the CSV file
    
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))  # Read the CSV file into a DataFrame
    return df  # Return the DataFrame containing the gate and qubit information


In [4]:
def unique(list1):
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if it exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)  # add the element to unique_list if it is not already present
    
    return unique_list  # return the list of unique elements


In [5]:
def depthFinder(df):
    qubits = df['Qubit'].value_counts()  # Count the occurrences of each qubit in the 'Qubit' column of the DataFrame
    qubits = dict(qubits)  # Convert the value counts to a dictionary
    lst = list(qubits.items())  # Convert the dictionary items to a list of tuples
    
    # Get depth
    if len(lst) > 1:  # If there are more than 1 qubit types in the circuit
        qnum = []
        
        # Extract the qubit number from each qubit type
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        
        qnum = unique(qnum)  # Get the unique qubit numbers
        
        counts = []
        
        # Count the number of occurrences of each unique qubit number
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        
        return len(counts)  # Return the total count of qubit numbers as the depth
        
    elif len(lst) > 0:  # If there is only 1 qubit type in the circuit
        return lst[0][1]  # Return the count of the single qubit type as the depth
    
    else:  # If there are no qubit types in the circuit
        return 0  # Return 0 as the depth


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import tree
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import pickle
# import the regressor
from sklearn.tree import DecisionTreeRegressor 

In [7]:
dataFile = "QMachineData//PerthDataSim.csv"
df = pd.read_csv(dataFile)

In [8]:
# headerList = ['Cost', 'Gate Count', 'Optimization Level', 'Runtime', 'Transpile Time']
  
# # converting data frame to csv
# df.to_csv("QMachineData//PerthDataSim.csv", header=headerList, index=False)
  

In [20]:
qc = QuantumCircuit(7)  # Create a quantum circuit with 7 qubits
qc.x(0)  # Apply a Pauli-X gate to qubit 0
qc.y(6)  # Apply a Pauli-Y gate to qubit 6
qc.h(1)  # Apply a Hadamard gate to qubit 1
qc.h(2)  # Apply a Hadamard gate to qubit 2
qc.h(2)  # Apply another Hadamard gate to qubit 2
qc.z(0)  # Apply a Pauli-Z gate to qubit 0
qc.measure_all()  # Measure all qubits in the circuit

start_time = time.time()  # Record the start time
tqc =  transpile(qc, backend, optimization_level=1)  # Transpile the circuit with optimization level 1
TT = time.time() - start_time  # Calculate the transpilation time
result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()  # Execute the transpiled circuit on the QASM simulator
RT = result.time_taken  # Get the total execution time
tempCost, tempGC = ComplexityFinder(tqc)  # Calculate the cost and depth of the transpiled circuit
print([tempCost, tempGC, 1, RT, TT])  # Print the cost, depth, optimization level, execution time, and transpilation time
Set1 = [tempCost, tempGC, 1, TT]  # Store the cost, depth, optimization level, and transpilation time in Set1

start_time = time.time()  # Record the start time
tqc =  transpile(qc, backend, optimization_level=2)  # Transpile the circuit with optimization level 2
TT = time.time() - start_time  # Calculate the transpilation time
result = execute(tqc, Aer.get_backend('qasm_simulator'), coupling_map=coupling_map, basis_gates=basis_gates, noise_model=noise_model).result()  # Execute the transpiled circuit on the QASM simulator
RT = result.time_taken  # Get the total execution time
tempCost, tempGC = ComplexityFinder(tqc)  # Calculate the cost and depth of the transpiled circuit
print([tempCost, tempGC, 2, RT, TT])  # Print the cost, depth, optimization level, execution time, and transpilation time
Set2 = [tempCost, tempGC, 2, TT]  # Store the cost, depth, optimization level, and transpilation time in Set2


[7, 7, 1, 0.07748937606811523, 0.009499073028564453]
[7, 7, 2, 0.08852291107177734, 0.016473770141601562]


In [21]:
#Linear Regression

In [22]:
X = np.asarray(df[['Cost', 'Gate Count', 'Optimization Level', 'Transpile Time']])
Y = np.asarray(df['Runtime'])
reg = LinearRegression().fit(X, Y)
print(reg.predict([Set1]))
print(reg.predict([Set2]))
print(reg.score(X,Y))
# filename='MLModels//PerthSimRegModel.sav'
# pickle.dump(reg, open(filename, 'wb'))

[0.10350676]
[0.09914812]
0.06579396322423858


In [23]:
#Decision Tree Regression

In [24]:
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X, Y)
print(reg.predict([Set1]))
print(reg.predict([Set2]))
print(reg.score(X,Y))

[0.10350676]
[0.09914812]
0.06579396322423858


In [12]:
correlation_matrix = df.corr()
correlation_matrix["Runtime"]

Cost                  0.155269
Gate Count            0.065725
Optimization Level   -0.199696
Runtime               1.000000
Transpile Time       -0.076048
Name: Runtime, dtype: float64